In [66]:
### Library Import Initialization

import numpy as np
import math
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [67]:
### Function to Import Stock Tickers and Calculate Final Stock Price

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
S_0

[*********************100%%**********************]  1 of 1 completed


135.24000549316406

In [68]:
### Sigma Calculation 

def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

sigma = compute_sigma(stock_data)
sigma


GOOG    0.326099
dtype: float64

In [74]:
### Function to Compute the Binomial Expected Value 

def binomial_model(S_0, K, T, r, sigma, option_type = 'call', n = 1000):
    """
    Parameters:
    S_0 (float): Current stock price (starting node of tree)
    K (float): Strike price of the option
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock
    option_type (str): Type of option ('call' or 'put')
    n (int): Number of steps in the binomial tree
    Returns:
    float: Option price
    """
    # Compute Delta T
    dt = T / n
    # Compute up and down parameters
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))
    # Compute probability of Increase
    p = (np.exp(r * dt) - d) / (u - d)

    # Init arrays for strike prices and option values
    S_n = np.zeros(n + 1)
    option_values = np.zeros(n + 1)

    # Generate the stock prices at each node S_n (points on Binomial Tree)
    ''' 
    S_n​ = S_0 * u^m * d^(n−m)
    m (int): Number of up movements from the initial node to node n, calculated as m = n - i
    '''
    for i in range(n+1):
        S_n[i] = S_0 * (u**(n - i)) * (d**(n - (n - i)))

    # Compute the option value (intrinsic value) at the final node
    for i in range(n+1):
        if option_type == 'call':
            option_values[i] = max((S_n[i] - K), 0)

        if option_type == 'put':    
            option_values[i] = max((K - S_n[i]), 0)
    #return option_values

    # Recursively iterate through the tree backwards to compute the option values at previous nodes
    # Outer  loop iterates over the time steps of the binomial tree in reverse order, starting from the last time step (n) and ending at the first time step (1)
    for j in range(n, 0, -1):
    # Inner loop iterates over each node at each time step
        for i in range(j):
            option_values[i] = (np.exp(r * dt) * ((p * option_values[i+1]) + (1-p) * option_values[i]))
                            
    # Return the first value of the option values array, which is the current option price
    return option_values[0]

### Function Output
option_type = 'call'
option_val = binomial_model(S_0, K = 139, T = 1, r = 0.025, sigma = sigma, option_type = option_type, n = 100)
print('The value of the ' + option_type + ' option price is: $' + str(option_val))


The value of the call option price is: $23.212018603283063
